Libraries utiles :

In [1]:
import sys
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as lg
import matplotlib
import matplotlib.pyplot as plt
from cfd_common import *
from math import *

In [2]:
%matplotlib qt

In [3]:
plt.ion()

Choix du domaine et des paramètres de la discrétisation du problème :

In [4]:
##### Domain #####

# Domain size
Lx = 4.
Ly = 2.

# Physical grid size
Nx_phys = 50*2
Ny_phys = 50
Δx = Lx/Nx_phys
Δy = Ly/Ny_phys
ΔxΔy = Δx*Δy
# Numerical grid size, including ghost points
Nx = Nx_phys + 2
Ny = Ny_phys + 2

# Physical grid (x,y)
# Due to the "staggered-like" approach to boundary conditions with ghost points,
#  the border go through the middle of border cells
x = np.linspace(Δx/2, Lx-Δx/2, Nx_phys) 
y = np.linspace(Δy/2, Ly-Δy/2, Ny_phys)
[xx,yy] = np.meshgrid(x,y) 

Initialisation des champs de vitesse / champ de pression et champ de densité :

On note $u,v$ les composantes horizontale, verticales du champ de vitesse. On les initialise à :...

In [5]:
##### Fields #####

# Velocity field (u,v)[i,j], initialized to "zero"
u = np.zeros((Nx,Ny))
v = np.zeros((Nx,Ny))

# Pressure field press[i,j] and its gradient (dx_press,dy_press)
press = np.zeros((Nx,Ny))
dx_press = np.zeros((Nx,Ny))
dy_press = np.zeros((Nx,Ny))

# Density field
ρ = np.ones((Nx,Ny))
ρ[:, 4*Ny//10 : 8*Ny//10] = 2

On impose les conditions aux bords (sur les GhostPoints) :

In [6]:
def VelocityGhostPoints(u,v):
    # ya pas cette histoire de "frontière = au milieu des cellules de bord ?"
    ### left
    u[0, :] = u[1, :]
    v[0, :] = v[1, :]
    ### right    
    u[-1, :] = u[-2, :]    # outflow condition => dérivée nulle
    v[-1, :] = v[-2, :]    # outflow condition => dérivée nulle
    ### bottom
    u[:,  0] = 0  # no slip
    v[:,  0] = 0  # imperméabilité
    ### top     
    u[:, -1] = 0#u[:, -2]
    v[:, -1] = 0#v[:, -2]
    
def PhiGhostPoints(phi):
    ### left
    phi[0,  :] = phi[1,  :]
    ### right             
    phi[-1, :] = -phi[-2, :]
    ### bottom               
    phi[:,  0] = phi[:,  1]*0
    ### top            
    phi[:, -1] = phi[:, -2]*0

def DensityGhostPoints(ρ):
    # ghost points for density : Neumann ∂ρ=0 boundary condition
    ρ[ 0, :] = ρ[1, :] # left
    ρ[-1, :] = ρ[-2, :] # right
    ρ[:,  0] = ρ[:, 1] # bottom
    ρ[:, -1] = ρ[:, -2] # top  

Ensuite vient la définition de notre schéma :

In [7]:
###### CONSTRUCTION des matrices et LU decomposition

### Matrix construction for projection step
LAPN = FD_2D_Laplacian_matrix (Nx_phys, Ny_phys, Δx, Δy, BCdir_left=False, BCdir_right=True, BCdir_top=False, BCdir_bot=False) 
LUPN = LUdecomposition(LAPN)

In [8]:
## Check mass
def mass_check(rho):
    return np.sum(rho)

mass_check(ρ)

7446.0

Puis la boucle principale de calcul :

In [9]:
### ATTENTION: dt_init calculer la CFL a chaque iteration... 
Δt = 0.0001

t = 0. # total time

# parameters (Reynolds number)
Re = 10


fig = plt.figure(1, figsize=(14,7))

loop_continue = True
def on_close (event):
    global loop_continue
    loop_continue = False
fig.canvas.mpl_connect('close_event', on_close)


niter = 0
while loop_continue:

    ###### Advection semi-Lagrangienne
    adv_u = SemiLag2(u,v, u, Δx,Δy,Δt)
    adv_v = SemiLag2(u,v, v, Δx,Δy,Δt)
    
    ρ = SemiLag2(u,v, ρ, Δx,Δy,Δt)
    # c'est totalement instable avec SemiLag2, pourquoi ????
    
    ### Mise à jour des conditions limites pour la densité.
    DensityGhostPoints(ρ)

    ###### Diffusion step
    ustar = adv_u + Δt*Laplacian(u, Δx,Δy)/Re/ρ
    vstar = adv_v + Δt*Laplacian(v, Δx,Δy)/Re/ρ
    
    g = 1
    α = pi/4
    ustar += +sin(α) * g * Δt
    vstar += -cos(α) * g * Δt
    
    ###### Mise à jour des conditions limites pour la vitesse :
    VelocityGhostPoints(ustar,vstar)
 
    ### Update divstar 
    divstar = Divergence(ustar,vstar, Δx,Δy)
    divstar = divstar - np.mean(divstar[1:-1,1:-1])
    
    ### Solving the linear system
    press[1:-1,1:-1] = Resolve(LUPN, RHS=divstar[1:-1,1:-1])

    ### Mise à jour des conditions limites pour la pression :
    press[0,  :] = press[1,  :] # left
    press[-1, :] = -press[-2, :] # right 
    press[:,  0] = press[:,  1] # bottom 
    press[:, -1] = press[:, -2] # top 

    ### Update gradphi
    dx_press[1:-1, :] = (press[2:, :] - press[:-2, :])/Δx/2
    dy_press[:, 1:-1] = (press[:, 2:] - press[:, :-2])/Δy/2

    ### Project u
    u = ustar - dx_press / ρ
    v = vstar - dy_press / ρ

    VelocityGhostPoints(u,v)
    
    
    

    if (niter%1000 == 0):
        #print(mass_check(ρ))
        fig.clear()
        (ax1,ax2) = fig.subplots(nrows=1, ncols=2, sharey=True)
        
        def plot_vel (ax, modx=1, mody=1, vmax=10):
            ax.quiver(xx[::modx,::mody], yy[::modx,::mody], np.transpose(u[1:-1:modx,1:-1:mody]), np.transpose(v[1:-1:modx,1:-1:mody]), scale=vmax*2)
            ax.set_aspect('equal', adjustable='box')
    
        fig.suptitle(r"Velocity, Density and Pression fields at $t={:.4f}$".format(t))
        
        ax1.clear()
        ax1.imshow(np.transpose(ρ[1:-1,1:-1]), origin='lower', extent=(0,Lx,0,Ly), cmap=plt.cm.RdPu, vmin=0, vmax=6)
        plot_vel(ax1)
        
        ax2.clear()
        im = ax2.imshow(np.transpose(press[1:-1,1:-1]), origin='lower', extent=(0,Lx,0,Ly), cmap='plasma')
        fig.colorbar(im)
        plot_vel(ax2)
        
#        my_cmap = plt.cm.gnuplot2_r(np.arange(plt.cm.gnuplot2_r.N))
#        my_cmap = 0.5 * (1+my_cmap)
#        my_cmap = matplotlib.colors.ListedColormap(my_cmap)
#        ax1.imshow(np.transpose(np.sqrt(u[1:-1,1:-1]**2 + v[1:-1,1:-1]**2)), origin='lower', extent=(0,Lx,0,Ly), cmap=my_cmap, vmin=0, vmax=2)
#        plot_vel(ax1)
    

        fig.tight_layout()
#       plt.savefig('blah/{:04d}.png'.format((niter+1)//modulo))
        plt.draw()
        
        mpl_pause_background(0.00001)
        
    t += Δt
    niter += 1

C:\Users\My-pr\Desktop\avalanche-simul-main\cfd_common.py:98: RuntimeWarning: overflow encountered in multiply
  Cc * q[1:-1, 1:-1]
C:\Users\My-pr\Desktop\avalanche-simul-main\cfd_common.py:99: RuntimeWarning: overflow encountered in multiply
  + Ce * ( Mx1*My1 * q[2: , 2: ]
C:\Users\My-pr\Desktop\avalanche-simul-main\cfd_common.py:98: RuntimeWarning: invalid value encountered in add
  Cc * q[1:-1, 1:-1]
C:\Users\My-pr\Desktop\avalanche-simul-main\cfd_common.py:103: RuntimeWarning: overflow encountered in multiply
  + Cmx * ( My1 * q[1:-1, 2:]
C:\Users\My-pr\Desktop\avalanche-simul-main\cfd_common.py:98: RuntimeWarning: overflow encountered in add
  Cc * q[1:-1, 1:-1]
C:\Users\My-pr\Desktop\avalanche-simul-main\cfd_common.py:105: RuntimeWarning: overflow encountered in multiply
  + Cmy * ( Mx1 * q[2:, 1:-1]
C:\Users\My-pr\Desktop\avalanche-simul-main\cfd_common.py:152: RuntimeWarning: invalid value encountered in subtract
  (u[2:, 1:-1] - u[:-2, 1:-1]) /Δx/2
C:\Users\My-pr\Desktop\aval

Traceback (most recent call last):
  File "C:\ProgramData\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-323e15b6a8a9>", line 26, in <module>
    ρ = SemiLag2(u,v, ρ, Δx,Δy,Δt)
  File "C:\Users\My-pr\Desktop\avalanche-simul-main\cfd_common.py", line 120, in SemiLag2
    adv_q = SemiLag(u,v, (3*q-qback)/2, Δx,Δy, Δt)
  File "C:\Users\My-pr\Desktop\avalanche-simul-main\cfd_common.py", line 90, in SemiLag
    Cc = (Δx - au*Δt) * (Δy - av*Δt) / ΔxΔy
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2045, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Trace

TypeError: object of type 'NoneType' has no len()